In [19]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from PyCorrelationMatrixManager.correlation_matrix import *

import os
import sys
import copy

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))

sys.path.append('..')
sys.path.append(os.path.join(os.path.expanduser('~'), "Code", "LQCD", "contraction_optimizer_pymodule"))

import ContractionOptimizerCPP as CO

## Operators

Operators for SU(4) baryons are constructed in full by Kimmy Cushman & Aaron Meyer.  For now they are hardcoded into the correct objects for the WickContraction library.  Specifically at the moment the below code sets up a list of "Elemental Operators" for uuuu operators, in different irreps, with a gamma matrix name and spatial coordinate.  

In [44]:
# load definition of the building blocks of the operators.
from src.baryon_elementals import BBsink, BBsource
from WickContractions.ops.operator import Operator
import numpy as np

aOps=[] # annihilation operators
cOps=[] # creation operators

aOps.append(BBsink({'uuuu': 1},{'uuuu': 1},'[000]','[000]',"\\Gamma^{{A1g0}}","\\Gamma^{{A1g0}}"))
cOps.append(BBsource({'uuuu': 1},{'uuuu': 1},'[000]','[000]',"\\Gamma^{{A1g0}}","\\Gamma^{{A1g0}}"))
#aOps.append(BBsink({'uuuu': 1},{'uuuu': 1},'[100]','[-100]',"\\Gamma^{{A1g0}}","\\Gamma^{{A1g0}}"))
#cOps.append(BBsource({'uuuu': 1},{'uuuu': 1},'[100]','[-100]',"\\Gamma^{{A1g0}}","\\Gamma^{{A1g0}}"))
    
    #TODO: multiflavor - something like below?
    #aOps.append(Operator([baryonSink([1,-1,+1,0.5],[['d','u','u','u'],['u','d','u','u'],...],'x_0',gammaName(g[0],g[1]))]))

In [45]:
for o in aOps:
    pprint(o)

<IPython.core.display.Math object>

In [46]:
for o in cOps:
    pprint(o)

<IPython.core.display.Math object>

## Correlation Matrix Manager

We are doing things by hand to use the contraction_optimizer

In [47]:
corrMatrix = CorrelationMatrix(
    cops=cOps, aops=aOps, 
    dts=[t for t in range(32)], t0s=[t for t in range(32)], 
    cfg=335)

In [48]:
corrMatrix.contract()
corrMatrix.laphify()

In [49]:
t,ctst=corrMatrix.correlators[0].diagrams[0].as_graph()
for i in t:
    print(i, "  ", t[i])
print(ctst)

0    B([000],t_i,\Gamma^{{A1g0}})
1    B([000],t_i,\Gamma^{{A1g0}})
2    B^*([000],t_i,t_f,\Gamma^{{A1g0}})
3    B^*([000],t_i,t_f,\Gamma^{{A1g0}})
{'3|1': [[3, 3], [2, 2], [1, 1], [0, 0]], '2|0': [[3, 3], [2, 2], [1, 1], [0, 0]]}


In [50]:
def contraction_as_COgraph(contractions):
    data=[]
    for cname, cindices in contractions.items():
        [l,r] = cname.split('|')
        data.append(( (int(l),int(r)), set(tuple(t) for t in cindices) ))

    return CO.Graph(data)

In [51]:
contraction_as_COgraph(ctst).getContractionList()

{(0, 2): {(0, 0), (1, 1), (2, 2), (3, 3)},
 (1, 3): {(0, 0), (1, 1), (2, 2), (3, 3)}}

In [52]:
len(corrMatrix.get_all_diagrams())

40320

In [53]:
global_tensors={}
global_tidx=0   
all_diagrams=[]
duplicate_tensors_fix=[]

# for the temporary fix, we have to through all the diagrams twice
# first we have to get the global tensors list
for i,d in enumerate(corrMatrix.get_all_diagrams()):
    tensors, contractions = d.as_graph()
    
    for t in tensors.values():
        if t not in global_tensors:
            global_tensors[t]=global_tidx
            global_tidx+=1

for tensor in global_tensors.copy().keys():
    global_tensors[tensor+"-B"]=global_tidx
    global_tidx+=1

for i,d in enumerate(corrMatrix.get_all_diagrams()):
    tensors, contractions = d.as_graph()

    cograph = contraction_as_COgraph(contractions)

    gtensors=[]
    duplicate_tensors_fix.append({})
    for t in tensors.values():
        if global_tensors[t] in gtensors:
            gtensors.append(global_tensors[t+"-B"])
        else:
            gtensors.append(global_tensors[t])

    #print(gtensors)
    all_diagrams.append(CO.Diagram(cograph,gtensors))
    

In [54]:
CO.ContractionCost.setDilutionRange(16)
optimizer = CO.ContractionOptimizer(all_diagrams)
optimizer.tune()

In [55]:
len(optimizer.getCompStepList())

39792

In [56]:
global_tensors

{'B([000],t_i,\\Gamma^{{A1g0}})': 0,
 'B^*([000],t_i,t_f,\\Gamma^{{A1g0}})': 1,
 'B([000],t_i,\\Gamma^{{A1g0}})-B': 2,
 'B^*([000],t_i,t_f,\\Gamma^{{A1g0}})-B': 3}

In [57]:
global_tensors_inv = {v: k for k,v in global_tensors.items()}
global_tensors_inv

{0: 'B([000],t_i,\\Gamma^{{A1g0}})',
 1: 'B^*([000],t_i,t_f,\\Gamma^{{A1g0}})',
 2: 'B([000],t_i,\\Gamma^{{A1g0}})-B',
 3: 'B^*([000],t_i,t_f,\\Gamma^{{A1g0}})-B'}

In [62]:
print("Contractions | GlobalTensor | ResultTensorIdx")
for c in optimizer.getCompStepList()[0:10]:
    #gt0=c[1][0]
    #gt1=c[1][1]
    #t0name=global_tensors_inv[gt0]
    #if t0name[-2:]=="-B":
    #    gt0=global_tensors[t0name[:-2]]
    #t1name=global_tensors_inv[gt1]
    #if t1name[-2:]=="-B":
    #    gt1=global_tensors[t1name[:-2]]
    
    print("{:<30} | {:^8} | {:^5}".format(str(CO.Graph.decodeElement(c[0])), str(c[1]), str(c[2])))

Contractions | GlobalTensor | ResultTensorIdx
{(1, 1), (3, 3), (2, 2), (0, 0)} |  (0, 1)  |   4  
{(1, 1), (3, 3), (2, 2), (0, 0)} |  (2, 3)  |   5  
{(0, 1), (1, 0), (3, 3), (2, 2)} |  (2, 3)  |   6  
{(1, 0), (0, 2), (3, 3), (2, 1)} |  (2, 3)  |   7  
{(3, 3), (1, 2), (2, 1), (0, 0)} |  (2, 3)  |   8  
{(0, 1), (1, 2), (2, 0), (3, 3)} |  (2, 3)  |   9  
{(1, 1), (0, 2), (3, 3), (2, 0)} |  (2, 3)  |  10  
{(1, 1), (0, 3), (2, 0), (3, 2)} |  (2, 3)  |  11  
{(0, 1), (3, 2), (1, 3), (2, 0)} |  (2, 3)  |  12  
{(3, 2), (1, 3), (2, 1), (0, 0)} |  (2, 3)  |  13  


In [16]:
for d in optimizer.getDiagramList()[0:10]:
    print(d.getResultIdList())

[4, 5]
[5, 6]
[5, 7]
[5, 8]
[5, 9]
[5, 10]
[5, 11]
[5, 12]
[5, 13]
[5, 14]
